### Packages

In [1]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

### Loading the data

In [2]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2018-01-29", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True)

[*********************100%***********************]  4 of 4 completed


In [5]:
raw_data

^FTSE                                                      \
                   Open         High          Low        Close       Volume   
Date                                                                          
1994-01-07  3401.399902  3446.800049  3398.699951  3446.000000          0.0   
1994-01-10  3465.699951  3468.100098  3430.000000  3440.600098          0.0   
1994-01-11  3442.500000  3442.500000  3413.500000  3413.800049          0.0   
1994-01-12  3394.800049  3402.399902  3372.000000  3372.000000          0.0   
1994-01-13  3380.699951  3383.300049  3356.899902  3360.000000          0.0   
...                 ...          ...          ...          ...          ...   
2018-01-22  7730.799805  7739.399902  7703.700195  7715.399902  730171300.0   
2018-01-23  7715.399902  7745.200195  7710.000000  7731.799805  742420200.0   
2018-01-24  7731.799805  7732.000000  7643.399902  7643.399902  746643300.0   
2018-01-25  7643.399902  7662.399902  7608.500000  7615.799805  713251600.0   
2018-01-26  7615.799805  7667.399902  7615.799805  7665.500000  713831300.0   

                  ^GSPC                                                       \
                   Open         High          Low        Close        Volume   
Date                                                                           
1994-01-07   467.089996   470.260010   467.029999   469.899994  3.249200e+08   
1994-01-10   469.899994   475.269989   469.549988   475.269989  3.194900e+08   
1994-01-11   475.269989   475.279999   473.269989   474.130005  3.054900e+08   
1994-01-12   474.130005   475.059998   472.140015   474.170013  3.106900e+08   
1994-01-13   474.170013   474.170013   471.799988   472.470001  2.779700e+08   
...                 ...          ...          ...          ...           ...   
2018-01-22  2809.159912  2833.030029  2808.120117  2832.969971  3.489480e+09   
2018-01-23  2835.050049  2842.239990  2830.590088  2839.129883  3.541140e+09   
2018-01-24  2845.419922  2852.969971  2824.810059  2837.540039  4.079270e+09   
2018-01-25  2846.239990  2848.560059  2830.939941  2839.250000  3.873100e+09   
2018-01-26  2847.479980  2872.870117  2846.179932  2872.870117  3.474680e+09   

                   ^N225                                            \
                    Open          High           Low         Close   
Date                                                                 
1994-01-07  17842.980469  18131.410156  17787.480469  18124.009766   
1994-01-10  18186.519531  18567.060547  18186.519531  18443.439453   
1994-01-11  18481.849609  18671.669922  18373.039062  18485.250000   
1994-01-12  18447.339844  18807.080078  18301.929688  18793.880859   
1994-01-13  18770.380859  18823.380859  18548.750000  18577.259766   
...                  ...           ...           ...           ...   
2018-01-22  23797.839844  23816.330078  23697.810547  23816.330078   
2018-01-23  23924.400391  24129.339844  23916.019531  24124.150391   
2018-01-24  24026.429688  24072.769531  23917.140625  23940.779297   
2018-01-25  23750.650391  23828.400391  23649.029297  23669.490234   
2018-01-26  23757.339844  23797.960938  23592.279297  23631.880859   

                              ^GDAXI                              \
                Volume          Open          High           Low   
Date                                                               
1994-01-07         0.0   2218.959961   2227.639893   2201.820068   
1994-01-10         0.0   2231.840088   2238.010010   2222.000000   
1994-01-11         0.0   2225.429932   2235.610107   2225.179932   
1994-01-12         0.0   2227.120117   2227.790039   2182.060059   
1994-01-13         0.0   2171.500000   2183.709961   2134.100098   
...                ...           ...           ...           ...   
2018-01-22  65200000.0  13435.139648  13470.049805  13410.959961   
2018-01-23  73400000.0  13577.139648  13596.889648  13517.809570   
2018-01-24  79900000.0  13572.709961  13576.839844  13414.740

In [3]:
df_comp = raw_data.copy()

In [4]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [6]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

### Creating Returns

In [7]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

### Splitting the Data

In [8]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

### Fitting a Model

In [9]:
from pmdarima.arima import auto_arima

In [10]:
model_auto = auto_arima(df.ret_ftse[1:])

In [10]:
model_auto

ARIMA(order=(4, 0, 5), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [11]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5019
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -7882.776
Date:                Wed, 11 Jan 2023   AIC                          15785.552
Time:                        17:33:50   BIC                          15850.762
Sample:                    01-11-1994   HQIC                         15808.403
                         - 04-05-2013                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0120      0.082      0.148      0.883      -0.148       0.172
ar.L2         -0.6543      0.077     -8.457      0.000      -0.806      -0.503
ar.L3         -0.1628      0.071     -2.290      0.022      -0.302      -0.023
ar.L4          0.2014      0.074      2.712      0.007       0.056       0.347
ma.L1         -0.0357      0.081     -0.440      0.660      -0.195       0.123
ma.L2          0.6067      0.078      7.769      0.000       0.454       0.760
ma.L3          0.0621      0.068      0.908      0.364      -0.072       0.196
ma.L4         -0.1934      0.073     -2.649      0.008      -0.336      -0.050
ma.L5         -0.1052      0.010    -11.067      0.000      -0.124      -0.087
sigma2         1.3543      0.015     91.438      0.000       1.325       1.383
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              6354.63
Prob(Q):                              0.96   Prob(JB):                         0.00
Heteroskedasticity (H):               1.99   Skew:                            -0.20
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.50
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Important Arguments

In [12]:
model_auto = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample = int(len(df_comp)*0.2))


# !!! Important Note: In pdmarima v1.5.2, out_of_sample_size is replaced with out_of_sample, so make sure to use the latter!


# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [15]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                
=========================================================================================
Dep. Variable:                                 y   No. Observations:                 6276
Model:             SARIMAX(2, 0, 2)x(1, 0, 1, 5)   Log Likelihood               -6340.661
Date:                           Fri, 03 Jan 2020   AIC                          12705.322
Time:                                   10:34:50   BIC                          12786.256
Sample:                               01-10-1994   HQIC                         12733.364
                                    - 01-29-2018                                         
Covariance Type:                             opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0043      0.009     -0.500      0.617      -0.021       0.013
drift       -1.45e-06   2.72e-06     -0.533      0.594   -6.78e-06    3.88e-06
ret_spx        0.0958      0.006     16.947      0.000       0.085       0.107
ret_dax        0.5582      0.005    113.729      0.000       0.549       0.568
ret_nikkei     0.0702      0.004     16.547      0.000       0.062       0.079
ar.L1         -0.1932      0.101     -1.918      0.055      -0.390       0.004
ar.L2          0.5083      0.059      8.643      0.000       0.393       0.624
ma.L1          0.0863      0.101      0.855      0.393      -0.112       0.284
ma.L2         -0.5369      0.056     -9.535      0.000      -0.647      -0.427
ar.S.L5        0.0705      0.339      0.208      0.836      -0.595       0.736
ma.S.L5       -0.0987      0.339     -0.291      0.771      -0.764       0.567
sigma2         0.4404      0.004     99.131      0.000       0.432       0.449
===================================================================================
Ljung-Box (Q):                       84.27   Jarque-Bera (JB):             15325.76
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.54   Skew:                             0.25
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.64
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""